In [1]:
import numpy as np
import pandas as pd
import redis

In [2]:
hostname = 'redis-13055.c305.ap-south-1-1.ec2.cloud.redislabs.com'
port = 13055
password = 'fOz4pYOutbMZi3dHLk1bhfBSmMws6spu'

In [3]:
r = redis.StrictRedis(host = hostname,
                      port = port,
                      password = password)

In [4]:
r.ping()

True

## Loading Dataframe from npz

In [5]:
file_np = np.load('dataframe_SRM.npz',allow_pickle = True)
file_np.files


['arr_0', 'arr_1']

In [6]:
df = pd.DataFrame(file_np['arr_0'], columns = file_np['arr_1'])
df

,Name,Roll,Features
0,RETHVIK MENON,21BCE1197,"[0.6674871, 1.167889, -1.104369, 0.7670394, -0..."
1,MUKUNDH J,21BAI1133,"[1.4518241, 0.758605, -0.44208556, 1.3326924, ..."
2,SUDARSHAN R,21BAI1257,"[0.60976905, -0.72095156, -0.741123, 1.5745338..."


In [7]:
df['Name_Roll'] = df['Name'] + '@' + df['Roll']

In [8]:
df

,Name,Roll,Features,Name_Roll
0,RETHVIK MENON,21BCE1197,"[0.6674871, 1.167889, -1.104369, 0.7670394, -0...",RETHVIK MENON@21BCE1197
1,MUKUNDH J,21BAI1133,"[1.4518241, 0.758605, -0.44208556, 1.3326924, ...",MUKUNDH J@21BAI1133
2,SUDARSHAN R,21BAI1257,"[0.60976905, -0.72095156, -0.741123, 1.5745338...",SUDARSHAN R@21BAI1257


In [11]:
records = df[['Name_Roll','Features']].to_dict(orient = 'records')


In [12]:
len(records)

3

In [13]:
for record in records:
    name_roll = record['Name_Roll']
    vector = record['Features']
    # print(name_roll)

    # convert numpy array to bytes
    vector_bytes = vector.tobytes()
    r.hset(name = 'academy:register',key = name_roll,value = vector_bytes)

## Retrieve Data

In [14]:
retrieve_dict = r.hgetall(name = 'academy:register')
retrieve_series = pd.Series(retrieve_dict)
retrieve_series

b'MUKUNDH J@21BAI1133'         b'_\xd5\xb9?\xf03B?\nY\xe2\xbe\xaa\x95\xaa?4lb...
b'SUDARSHAN R@21BAI1257'       b'\xd3\x19\x1c?H\x908\xbf=\xba=\xbfS\x8a\xc9?P...
b' RETHVIK MENON@21BCE1197'    b'o\xe0*?c}\x95?\xf7[\x8d\xbf\xb2\\D?\x00\x03z...
dtype: object

In [15]:
np.frombuffer(retrieve_series[b'SUDARSHAN R@21BAI1257'],dtype = np.float32).shape

(512,)

In [16]:
retrieve_series = retrieve_series.apply(lambda x: np.frombuffer(x,dtype = np.float32))
index = retrieve_series.index
index = list(map(lambda x: x.decode(),index))

In [17]:
retrieve_series.index = index

In [18]:
retrieve_df = retrieve_series.to_frame().reset_index()
retrieve_df.columns = ["Name_Roll","Features"]
retrieve_df.head()

,Name_Roll,Features
0,MUKUNDH J@21BAI1133,"[1.4518241, 0.758605, -0.44208556, 1.3326924, ..."
1,SUDARSHAN R@21BAI1257,"[0.60976905, -0.72095156, -0.741123, 1.5745338..."
2,RETHVIK MENON@21BCE1197,"[0.6674871, 1.167889, -1.104369, 0.7670394, -0..."


## Registration form 

In [19]:
import cv2
from insightface.app import FaceAnalysis
faceapp = FaceAnalysis(name = 'buffalo_l',
                       root = 'InsightFaceModels',
                       providers = ['CPUExecutionProvider'])
faceapp.prepare(ctx_id = 0,det_size=(640,640),det_thresh = 0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: InsightFaceModels\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: InsightFaceModels\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: InsightFaceModels\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: InsightFaceModels\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: InsightFaceModels\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [20]:
# Name and Roll

name = str(input("Enter Name in CAPS"))
roll = str(input("Enter Roll NO in CAPS:"))

name_roll = name+'@'+roll
print(name_roll)



Enter Name in CAPS SUDARSHAN
Enter Roll NO in CAPS: 21BAI1257


SUDARSHAN@21BAI1257


In [21]:
face_embeddings = []
vid = cv2.VideoCapture(0) 
sample = 0 
while True: 
    ret, frame = vid.read() 

    if ret == False:
        print("unable to read camera")
        break
        
    # Display the resulting frame 
    
    
    frame_face = faceapp.get(frame,max_num = 1)
    
    for face in frame_face:
        sample += 1
        x1, y1, x2, y2 = face['bbox'].astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)

        embeddings = face['embedding']
        face_embeddings.append(embeddings)
    if sample >= 200:
        break

        
    cv2.imshow('frame', frame) 
	# the 'q' button is set as the  
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 


C:\Users\sudar\OneDrive\Desktop\AttendanceApp\attendanceapp\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [22]:
print(face_embeddings)

[array([-0.3814114 , -0.27265406, -0.42742437,  0.8373645 ,  0.09957313,
       -0.8104221 , -1.206475  , -0.709868  , -1.1493847 , -0.55178994,
        0.87009263, -1.1713414 , -1.0705675 ,  1.5693005 ,  0.11085199,
        0.43681583, -0.14292665,  0.25579855, -0.647401  , -0.2773878 ,
        0.5066566 ,  1.6068583 , -0.06395611,  1.1292338 , -0.93291175,
        0.37358418, -1.9852746 , -0.636786  ,  0.12855034, -0.14756237,
       -0.6378934 , -0.07467993,  1.1717792 ,  0.11512434,  0.6407066 ,
        0.7627824 , -0.27356642,  0.5896748 , -1.033911  , -1.8364469 ,
       -0.11715147,  0.290513  , -0.51509565,  0.19849706, -1.1054966 ,
        0.27614242, -0.51591796, -0.48201603, -0.90060633, -0.80741507,
        0.323894  ,  0.45365876,  1.9274938 ,  1.4188092 , -0.033671  ,
       -0.54090136,  0.15657935, -0.8739254 , -0.82320434,  1.9763882 ,
        0.66483927,  1.590771  , -1.471176  ,  2.037923  , -0.47090685,
        1.2932307 ,  1.0856081 ,  0.33913696, -1.3544967 , -0.4

In [23]:
len(face_embeddings)

10

## Storing in DB


In [24]:
x_mean = np.asarray(face_embeddings).mean(axis = 0)
x_mean.shape

(512,)

In [25]:
x_mean

array([ 5.76898932e-01,  2.30772257e-01, -5.69662690e-01,  1.16088367e+00,
       -2.24881321e-01, -1.47769406e-01, -1.90277135e+00, -5.02444208e-01,
       -8.84650886e-01, -5.36893964e-01,  9.15263176e-01, -7.62770534e-01,
       -5.57332523e-02,  2.21824670e+00, -2.10874110e-01, -9.34558570e-01,
        3.35112005e-01,  8.10714602e-01,  5.07171825e-02, -8.17192554e-01,
       -2.30096146e-01,  4.74222600e-01, -3.47236007e-01,  7.30894744e-01,
       -1.74937606e+00, -1.19939804e+00, -1.41990829e+00,  2.38333434e-01,
        5.89147568e-01, -1.19263843e-01, -6.07891381e-01,  8.96065682e-02,
        1.66816607e-01, -6.03741482e-02,  4.32137549e-01,  2.20793748e+00,
        7.77328849e-01,  4.14909899e-01, -6.36342645e-01, -1.62488770e+00,
       -1.03550363e+00, -5.43736041e-01, -3.83242071e-01,  1.17065823e+00,
       -1.76271453e-01, -2.75964797e-01,  2.03881022e-02, -1.50980085e-01,
       -5.96530080e-01,  3.40030432e-01, -3.35425049e-01,  7.88629591e-01,
        1.29793930e+00,  

In [26]:
x_mean = x_mean.tobytes()

In [27]:
r.hset(name = 'academy:register',key = name_roll,value = x_mean)

1